In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import cv2
import random
import os
import pandas as pd
from PIL import Image
from skimage import exposure
from scipy.ndimage import gaussian_filter
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight

2024-04-13 22:51:41.240733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 22:51:41.398433: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 22:51:42.213525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-13 22:51:42.213644: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import os

# Set environment variable for TensorFlow
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import tensorflow as tf

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-04-13 22:51:49.387649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 22:51:49.904564: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0


In [3]:
import numpy as np
import random
import tensorflow as tf
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [4]:
def rotate_image(image, angle):
    # Rotate the image by the specified angle
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    if random.choice([True, False]):
        image = cv2.flip(image, 1)
    if random.choice([True, False]):
        image = cv2.flip(image, 0)
    return image


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )  # Ensure matrix type is float32

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image


def augmented_fn(image):
    angle = random.uniform(-35, 35)
    image = rotate_image(image, angle)
    image = flip_image(image)
    image = shear_image(image, shear_range=0.15)
    image = translate_image(image, translate_range=0.1)
    image = adjust_brightness(image, brightness_range=0.25)

    return image


def apply_augmentation(image):
    augmented_images = []
    for _ in range(20):
        augmented_image = augmented_fn(image)
        augmented_images.append(augmented_image)
    return np.array(augmented_images)

In [8]:
# Load data from file
X_train_data = np.load(
    "/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/X_train.npy"
)
X_test_data = np.load("/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/X_test.npy")
y_train_data = np.load(
    "/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/y_train.npy"
)
y_test_data = np.load("/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/y_test.npy")


# Assign X_test data to X_train
X_train = X_train_data
X_test = X_test_data
y_train = y_train_data
y_test = y_test_data

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(512, 512, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the pre-trained model
model = Sequential(
    [
        base_model,
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(5, activation="softmax"),  # Output layer with 5 classes
    ]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # Use sparse categorical crossentropy for multi-class classification
    metrics=["accuracy"],
)

# Print model summary
model.summary()

# Calculate class weights manually
class_counts = np.bincount(y_train)
total_samples = np.sum(class_counts)
class_weights = {i: total_samples / count for i, count in enumerate(class_counts)}

# Print class weights
# print("Class Weights:", class_weights)


# Define callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_accuracy", save_best_only=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=25,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stopping, model_checkpoint],
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 16, 16, 512)       14714688  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 512)               67109376  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 81,826,629
Trainable params: 67,111,941
Non-trainable params: 14,714,688
____________________________________

2024-04-13 22:55:55.103475: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2024-04-13 22:55:55.218564: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


30/30 [==============================] - 15s 356ms/step - loss: 476.0885 - accuracy: 0.2796 - val_loss: 27.6922 - val_accuracy: 0.2232
Epoch 2/25
30/30 [==============================] - 9s 295ms/step - loss: 31.9178 - accuracy: 0.4312 - val_loss: 1.2998 - val_accuracy: 0.3605
Epoch 3/25
30/30 [==============================] - 9s 300ms/step - loss: 5.8389 - accuracy: 0.3430 - val_loss: 1.2471 - val_accuracy: 0.4335
Epoch 4/25
30/30 [==============================] - 9s 289ms/step - loss: 4.9319 - accuracy: 0.4011 - val_loss: 1.3513 - val_accuracy: 0.4893
Epoch 5/25
29/30 [============================>.] - ETA: 0s - loss: 4.5745 - accuracy: 0.4591